# Some useful tools

## Cut and output coast data

In [15]:
import subprocess

import numpy as np

latmin, latmax, lonmin, lonmax = 18, 21, -157, -155
out = "./coast"
is_comment = False

r = subprocess.run("gmt coast -M -R%.1f/%.1f/%.1f/%.1f -W" % (lonmin, lonmax, latmin, latmax), shell=True, capture_output=True, text=True)
lines = r.stdout.split('\n')
with open(out, 'w') as f:
    for line in lines:
        if line and (not is_comment) and (line[0]=='>'):
            continue
        else:
            f.write(line+'\n')

# SEM Tool functions

## Prepare topography

In [3]:
import pygmt
import numpy as np
from os.path import join

outdir = "/home/shijie/data/hawaii/sem_test/to_cluster/meshfem3D_files/"
fint = join(outdir, "interfaces.dat")
flay1 = join(outdir, "interface1.dat")
flay2 = join(outdir, "interface2.dat")
latmin, latmax, lonmin, lonmax = 18, 21, -157, -155
nlay1 = 15
nlay2 = 6

grid = pygmt.datasets.load_earth_relief(resolution='15s', region=[lonmin, lonmax, latmin, latmax])

lon = grid.coords['lon'].data
lat = grid.coords['lat'].data
Nx, Ny = len(lon), len(lat)
dx = lon[1] - lon[0]
dy = lat[1] - lat[0]
x0, y0 = lon[0], lat[0]

elev = -np.ravel(grid.data, order='C')
#elev = np.zeros(np.ravel(grid.data, order='C').shape)
np.savetxt(flay2, elev, fmt="%.7e")

layer1 = elev + np.ones(elev.shape) * (-10000)
np.savetxt(flay1, layer1, fmt="%.7e")

with open(fint, 'w') as f:
    f.write("2\n") # number of interfaces
    f.write(".false. %d %d %.7f %.7f %.7f %.7f\n" % (Nx, Ny, x0, y0, dx, dy))
    f.write("interface1.dat\n")
    f.write(".false. %d %d %.7f %.7f %.7f %.7f\n" % (Nx, Ny, x0, y0, dx, dy))
    f.write("interface2.dat\n")
    f.write("%d\n%d\n" % (nlay1, nlay2))